## 

# <center>Consultas ao SGS do Banco Central<center>

___

* Now
    * Adicionar coluna com apelido para filtro inicial
    
* Next    
    * Montar acumulado
    * Montar demonstrativo do balanço de pagamentos
    * Curva de juros

___

## Imports

In [1]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import datetime as dt

import datetime
from datetime import date

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sgs

___

## Consultas SGS

In [2]:
# SGS func
def sgs_bc(cod, start, end):
    return sgs.time_serie(cod, start = start, end = end).to_frame()

In [3]:
# consulta códigos
def cod_bc(text):
    results = sgs.search_ts(text, language = "pt")
    results = pd.DataFrame(results)
    #results = results.loc[results['last_value'] >= '01/01/2020']
    
    return results

___

## Import data

In [4]:
# Import data
CodBC = pd.read_csv('data/CodBC.csv')

In [5]:
# Fillna
CodBC.fillna('', inplace = True)

# Concat columns
CodBC['complete_names'] = CodBC['name'] + ' ' + CodBC['description'] + ' ' + CodBC['detail'] + ' - ' + CodBC['code'].map(str)

## Tratar textos

In [6]:
text_columns = ['name', 'description', 'detail', 'complete_names']

In [7]:
# Capitalize
for col in text_columns:
    CodBC[col] = CodBC[col].str.capitalize()
    
# Upper
for col in text_columns:
    CodBC[col] = CodBC[col].str.replace(' us ', ' US ')
    CodBC[col] = CodBC[col].str.replace(' rs ', ' RS ')
    CodBC[col] = CodBC[col].str.replace(' pib ', ' PIB ')
    CodBC[col] = CodBC[col].str.replace('pib', 'PIB')
    CodBC[col] = CodBC[col].str.replace(' selic ', ' SELIC ')
    CodBC[col] = CodBC[col].str.replace('selic', 'SELIC')
    CodBC[col] = CodBC[col].str.replace('Selic', 'SELIC')
    CodBC[col] = CodBC[col].str.replace(' ipca ', ' IPCA ')
    CodBC[col] = CodBC[col].str.replace('ipca', 'IPCA')
    CodBC[col] = CodBC[col].str.replace(' inpc ', ' INPC ')
    CodBC[col] = CodBC[col].str.replace('inpc', 'INPC')
    CodBC[col] = CodBC[col].str.replace(' ibge ', ' IBGE ')
    CodBC[col] = CodBC[col].str.replace('ibge', 'IBGE')
    CodBC[col] = CodBC[col].str.replace('cambio', 'câmbio')
    CodBC[col] = CodBC[col].str.replace('Cambio', 'Câmbio')
    CodBC[col] = CodBC[col].str.replace('Tjlp', 'TJLP')
    
# Adjusts names, description and detail
CodBC.loc[CodBC['name'].isin(['Índice nacional de preços ao consumidor (INPC)']), 'detail'] = CodBC.loc[CodBC['name'].isin(['Índice nacional de preços ao consumidor (INPC)']), 'description']
CodBC.loc[CodBC['name'].isin(['Índice nacional de preços ao consumidor (INPC)']), 'description'] = ('INPC')
CodBC.loc[CodBC['name'].isin(['Índice de nacional de preços ao consumidor']), 'name'] = 'Índice nacional de preços ao consumidor'
CodBC.loc[CodBC['description'] == '', 'description'] = CodBC.loc[CodBC['description'] == '', 'name']
CodBC.loc[CodBC['detail'] == '', 'detail'] = CodBC.loc[CodBC['detail'] == '', 'description']
CodBC.loc[CodBC['name'].isin(['Transações correntes acumulado em 12 meses em relação ao pib']), 'name'] = 'Transações correntes acumulado em 12 meses/pib'

___

## Plot func

In [8]:
def plot(df):
    
    # Tick colors
    tick_color = '#485656'

    fig = go.Figure()

    # Add traces
    for col in df:
        
        fig.add_trace(go.Scatter(x = df.index, y=df[col],
                                 marker = dict(size = 1.75),
                                 mode='lines',
                                 name=col))
    
    fig.update_layout(template = 'plotly_white')
    fig.update_xaxes(showspikes = True, spikemode = 'across', spikesnap = 'cursor', spikedash = 'dot', spikethickness = 0.35)
    fig.update_yaxes(showspikes = False, spikemode = 'across', spikesnap = 'cursor', spikedash = 'dot', spikethickness = 0.35)
    fig.update_xaxes(title = '', showgrid = False, gridwidth = None, gridcolor = None, showline = True, linecolor = 'dimgrey', linewidth = 1.4)                
    fig.update_yaxes(title = None, showgrid = True, gridwidth = None, gridcolor = 'rgba(220,220,220,0.2)', showline = False, linecolor = 'white', linewidth = 1, color = tick_color)
    
    fig.update_layout(modebar_add = ['v1hovermode', 'toggleSpikelines'], hovermode = 'x') #closest
    fig.update_layout(paper_bgcolor = '#FEFEFE', plot_bgcolor = '#FEFEFE')
    fig.update_layout(title = '', title_font_size = 22, title_x = 0.5, template = 'plotly_white')
    
    fig.update_layout(autosize = True, width = None, height = 550, showlegend = True, 
                      legend = dict(font = dict(size = 13), orientation = 'h', y=-0.25, x=0.00, yanchor=None, xanchor = None))

    #fig.update_traces(textposition = 'top center')
    
    return fig

___

# App code

In [9]:
# Imports 
import dash
import dash_auth
from dash import dcc
from dash import html
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

In [10]:
# Top menu
TOP_MENU_STYLE = {
    "margin-left": "0rem",
    "margin-right": "0rem",
    "padding": ".35rem 0rem",
    'width': '100%', 
    "height": "3.5rem",
    "background-color": "#4A5959"
}

# padding for the page content
TITLE_STYLE = {
    "margin-left": "0rem",
    "margin-right": "0rem",
    "padding": "0.0rem 0rem",
    "background-color": "#f3f5f5",
}

CONTENT_STYLE = {
    "margin-left": "0rem",
    "margin-right": "0rem",
    "margin-bottom": "0rem",
    "padding": "0rem 2rem",
    "height": "38rem",
    "background-color": "#f3f5f5",
}

CARD_STYLE_SIDEBAR = {
    "margin-left": "0rem",
    "margin-right": "0rem",
    'margin-top': '0rem',
    'width': '90%', 
    'border-radius':'7px', 
    'border': '0px',
    'box-shadow': ' 3px 4px 6px rgba(219,219,219,0.8)',
    'background-color': '#FEFEFE',
    'height':'70vh'
}

CARD_STYLE = {
    "margin-left": "0rem",
    "margin-right": "0rem",
    'margin-top': '0px',
    'width': '100%', 
    'border-radius':'7px', 
    'border': '0px',
    'box-shadow': ' 3px 4px 6px rgba(219,219,219,0.8)',
    'background-color': '#FEFEFE'
}

CARD_GRAPH_STYLE = {
    "margin-left": "-3rem",
    "margin-right": "0rem",
    'margin-top': '0rem',
    'width': '100%', 
    'height':'70vh',
    'verticalAlign': 'center',
    'border-radius':'7px', 
    'border': '0px',
    'box-shadow': ' 3px 4px 6px rgba(219,219,219,0.8)',
    'background-color': '#FEFEFE'
}

# app code

In [11]:
# Set user and password
USERNAME_PASSWORD_PAIRS = [['username', 'password'],
                           ['Fernando', 'Senha#']]

# Initialise the app 
app = dash.Dash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP])

# server
server = app.server

# Autentication
auth = dash_auth.BasicAuth(app, USERNAME_PASSWORD_PAIRS)

########################################################################################################################

# Parameters
#today = dt.datetime.today().strftime('%d/%m/%Y')
last_date = '2021-01-01'
min_date = pd.to_datetime('2010-01-01').date()
max_date = pd.to_datetime(dt.date.today()).date()

########################################################################################################################

# Top menu
top_menu = html.Div(id = 'top_menu',
    children = [
        dbc.Row([
            dbc.Col([
                dbc.Col(html.P(
                    dbc.RadioItems(
                        id = "select_page",
                        className = "btn-group", #d-flex
                        inputClassName = "btn-check",
                        labelClassName = "btn outline-dark", #btn btn-outline-secondary
                        labelCheckedClassName = "active",
                        inline = True,
                        #inputStyle = {"backgroundColor": 'FDFDFD'},
                        options = [{"label": "Home", "value": 1}],
                        value = 3, 
                        style = {'margin-top': '0px',},
                        labelStyle = {'fontSize': 20, 'margin-top': '0x', 'margin-left': '0px', 'color': '#FDFDFD', 
                                     },
                        ),
                    ),  xs = 2, sm = 1, md = 12, lg = 12, xl = 2, align = 'center')
            ]),
            dbc.Col([
                dbc.Col(html.H2('', className = 'text-center lead', 
                                style = {'fontsize': 16, 'color': '#FDFDFD', 'margin-left': '0rem'}), 
                        xs = 2, sm = 1, md = 12, lg = 12, xl = 2, align = 'center')
            ]),
            dbc.Col([
                dbc.Col(html.H2('', className = 'text-center lead'), 
                        xs = 2, sm = 1, md = 12, lg = 12, xl = 6, align = 'center')
            ]),
        ])
    ], style = TOP_MENU_STYLE)

########################################################################################################################

# Title
title = html.Div(id = 'title',
                   children = [
                       dbc.Row([
                           dbc.Col(html.H2('Economics', className = 'text-center display-3'), 
                                   xs = 2, sm = 1, md = 12, lg = 12, xl = 12)
                       ]),
                       dbc.Row([
                           dbc.Col(html.H6('Economic Data History', className = 'text-center lead',
                                          style = {'fontSize': 30}), 
                                   xs = 1, sm = 1, md = 12, lg = 12, xl = 12)
                       ])
                   ], style = TITLE_STYLE)

########################################################################################################################

# Explorer   
content = html.Div(id = 'content', 
                   children = [
                       dbc.Row([
                           dbc.Col(
                               dbc.Card([
                                   dbc.CardBody([
                                       html.P('Period', className = 'lead', 
                                              style = {'fontSize': 20, 'margin-top': '0px', 'margin-left': '0px', 'margin-botton': '10px'}),
                                       
                                       # Data interval
                                       dcc.DatePickerRange(
                                           id = 'period',
                                           min_date_allowed = date(min_date.year, min_date.month, min_date.day),
                                           max_date_allowed = date(max_date.year, max_date.month, max_date.day),
                                           start_date = date(min_date.year, min_date.month, min_date.day),
                                           end_date = date(max_date.year, max_date.month, max_date.day),
                                           className = None,
                                           display_format = 'MMM Y',
                                           calendar_orientation = 'horizontal',
                                           style = {'fontSize': 16, 'margin-left': '35px'}),
                                       
                                       # Group by
                                       html.P('Acconts', className = 'lead', 
                                              style = {'fontSize': 20, 'margin-top': '15px', 'margin-left': '0px'}),
                                       
                                       dcc.Dropdown(id = 'names', multi = True, value = ['Taxa de câmbio'], clearable = True,
                                                    style = {'margin-left': '0px', 'margin-right': '0px', 'margin-top': '0px'},
                                                    options = [{'label': x, 'value': x} for x in sorted(list(CodBC.loc[CodBC['last_value'] >= last_date, 'name'].unique()))]),
                                       
                                       html.P('Description', className = 'lead', 
                                              style = {'fontSize': 20, 'margin-top': '15px', 'margin-left': '0px'}),
                                       
                                       dcc.Dropdown(id = 'description', multi = True, value = ['Livre'], clearable = True, 
                                                    style = {'margin-left': '0px', 'margin-right': '0px', 'margin-top': '0px'},
                                                    options = []),
                                       
                                       html.P('Append', className = 'lead', 
                                              style = {'fontSize': 20, 'margin-top': '15px', 'margin-left': '0px'}),
                                       
                                       dcc.Dropdown(id = 'detail', multi = True, value = ['Dólar americano (compra)'], clearable = True, 
                                                    style = {'margin-left': '0px', 'margin-right': '0px', 'margin-top': '0px'},
                                                    options = []),
                                       ])], 
                                       color = "light", outline = True, style = CARD_STYLE_SIDEBAR),
                               xs = 12, sm = 12, md = 12, lg = 12, xl = 4, align = "around"),
                                                      
                           dbc.Col([  
                               dbc.Row([
                                   dbc.Col(
                                       dbc.Card([
                                           dbc.CardBody([
                                               dcc.Loading([
                                                   dcc.Graph(id = 'graph', figure = {})
                                               ], color = '#119DFF', type = 'dot', fullscreen = False)

                                           ])],  color = "light", outline = True, style = CARD_GRAPH_STYLE), width = 12)
                               ], justify = 'around')
                           ], width = 8),
                       ], justify = 'around'),
                   ], style = CONTENT_STYLE)

# Define de app
app.layout = html.Div([
    top_menu,
    title,
    content
])


#########################################################################################################################

# callbacks

# Populate dropdown description with options and values from names
@app.callback(
    Output(component_id = 'description', component_property = 'options'),
    Input(component_id = 'names', component_property = 'value'),
    prevent_initial_call = False )

def setDescription(names):
    return [{'label': x, 'value': x} for x in sorted(list(CodBC.loc[(CodBC['last_value'] >= last_date) &
                                                                    (CodBC['name'].isin(names)), 'description'].unique()))]


# Populate dropdown description with options and values from names
@app.callback(
    Output(component_id = 'detail', component_property = 'options'),
    Input(component_id = 'description', component_property = 'value'),
    Input(component_id = 'names', component_property = 'value'),
    prevent_initial_call = False )

def setDescription(description, names):
    return [{'label': x, 'value': x} for x in sorted(list(CodBC.loc[(CodBC['last_value'] >= last_date) &
                                                                    (CodBC['name'].isin(names)) &
                                                                    (CodBC['description'].isin(description)), 
                                                                    'detail'].unique()))]

# Update Graph
@app.callback(
    Output('graph', 'figure'),
    Input('period', 'start_date'),
    Input('period', 'end_date'),
    Input('names', 'value'),
    Input('description', 'value'),
    Input('detail', 'value'),
    prevent_initial_call = False)   

def update_graph(start_date, end_date, names, description, detail):
    
    # Adjusts
    start_date = pd.to_datetime(start_date).date().strftime('%d/%m/%Y')
    end_date = pd.to_datetime(end_date).date().strftime('%d/%m/%Y')    
    
    # Make cod list
    def codList(last_date, names, description, detail):
        CodBC_filtered = CodBC.loc[(CodBC['last_value'] >= last_date ) & (CodBC['name'].isin(names)) & 
                                   (CodBC['description'].isin(description)) & (CodBC['detail'].isin(detail)),
                                  ['code', 'complete_names']]

        cod_list = []
        for code in CodBC_filtered['code']:
            cod_list.append([code, CodBC_filtered.loc[CodBC_filtered['code'] == code, 'complete_names'].sum()])

        return cod_list

    cod_list = codList(last_date, names, description, detail)
    
    # Table
    def buildTable(cod_list, start_date, end_date):
    
        df_consult = pd.DataFrame()
        for item in cod_list:
            df = pd.DataFrame(sgs_bc(int(item[0]), start_date, end_date)).rename(columns = {int(item[0]): item[1]})
            df_consult = pd.concat([df_consult, df], axis = 1)

        return df_consult

    df_consult = buildTable(cod_list, start_date, end_date)
    
    
    return plot(df_consult)
      
#########################################################################################################################

# Run the app
if __name__ == '__main__':
    app.run_server(debug = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2023 18:19:21] "GET / HTTP/1.1" 401 -
127.0.0.1 - - [19/Apr/2023 18:19:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:20:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:20:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:20:00] "GET /_favicon.ico?v=2.5.0 HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:20:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:20:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:20:00] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Apr/2023 18:20:00] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Apr/2023 18:20:00] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Apr/2023 18:20:00] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1

___